In [ ]:
import sys
# !{sys.executable} -m pip install tiktoken
# !{sys.executable} -m pip install pandas>=2.0.0
import os

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2"

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # for 1B
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
import tiktoken
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig,LlamaTokenizer
from datasets import load_dataset
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datetime import datetime

In [ ]:
from trl import SFTTrainer, setup_chat_format
import json
from peft import PeftModel, PeftConfig,AutoPeftModelForCausalLM
from transformers import MllamaForConditionalGeneration, AutoProcessor
import torch
torch_dtype = torch.float16
attn_implementation = "eager"
# !{sys.executable} -m pip install bitsandbytes
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)
import time

def load_model_tokenizer_questions(base_model, lora_model):
    
    tokenizer = AutoTokenizer.from_pretrained(base_model)
    model = MllamaForConditionalGeneration.from_pretrained(base_model, quantization_config=bnb_config, device_map="auto",attn_implementation=attn_implementation)
    model.resize_token_embeddings(len(tokenizer))  # Or len(tokenizer) if you're using the LoRA tokenizer
    model = PeftModel.from_pretrained(model, lora_model, device_map="auto")
    
    return model, tokenizer

In [ ]:
model, tokenizer = load_model_tokenizer_questions(
    base_model="meta-llama/Llama-3.2-11B-Vision-Instruct",
    lora_model="xfu20/BEMGPT_1"
)